In [2]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LEVX/LEVXfusion1.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LEVX/LEVXtexts_test1.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/gpt/LEVX/LEVXtokenizer_config1.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3026
Total X variables: 176072

Text train example1


'19005kt 9999 prec0n CL5 CM2 13 12 q1017 20008KT 170V230 3000 BR SCT003 BKN006 13/13 Q1016 TEMPO 0800 FG VV001'


Sequences example 1


[151,
 1,
 2,
 41,
 70,
 9,
 7,
 22,
 328,
 144,
 31,
 53,
 245,
 240,
 9,
 9,
 25,
 16,
 96,
 45,
 42]


Text train example2


...

[621, 1, 32, 41, 158, 11, 8, 46, 427, 503, 1, 489, 575, 9, 9, 43, 4]


X


array([[  0,   0,   0,   0,   0, 151,   1,   2,  41,  70,   9,   7,  22],
       [  0,   0,   0,   0, 151,   1,   2,  41,  70,   9,   7,  22, 328],
       [  0,   0,   0, 151,   1,   2,  41,  70,   9,   7,  22, 328, 144],
       [  0,   0, 151,   1,   2,  41,  70,   9,   7,  22, 328, 144,  31],
       [  0, 151,   1,   2,  41,  70,   9,   7,  22, 328, 144,  31,  53],
       [151,   1,   2,  41,  70,   9,   7,  22, 328, 144,  31,  53, 245],
       [  1,   2,  41,  70,   9,   7,  22, 328, 144,  31,  53, 245, 240],
       [  2,  41,  70,   9,   7,  22, 328, 144,  31,  53, 245, 240,   9],
       [ 41,  70,   9,   7,  22, 328, 144,  31,  53, 245, 240,   9,   9],
       [ 70,   9,   7,  22, 328, 144,  31,  53, 245, 240,   9,   9,  25],
       [  9,   7,  22, 328, 144,  31,  53, 245, 240,   9,   9,  25,  16],
       [  7,  22, 328, 144,  31,  53, 245, 240,   9,   9,  25,  16,  96],
       [ 22, 328, 144,  31,  53, 245, 240,   9,   9,  25,  16,  96,  45],
       [  0,   0,   0,   0,   0, 621, 


Dictionary first 10 words


{'9999': 1,
 'prec0n': 2,
 'cm0': 3,
 'nosig': 4,
 'cl0': 5,
 'cavok': 6,
 '12': 7,
 '11': 8,
 '13': 9,
 '10': 10}

In [3]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/gpt/LEVX/LEVXmodel1.keras")


Epoch 1/50
344/344 [==============================] - 26s 67ms/step - loss: 3.8154 - accuracy: 0.2225
Epoch 2/50
344/344 [==============================] - 23s 67ms/step - loss: 2.7691 - accuracy: 0.3146
Epoch 3/50
344/344 [==============================] - 23s 67ms/step - loss: 2.5139 - accuracy: 0.3421
Epoch 4/50
344/344 [==============================] - 23s 67ms/step - loss: 2.3821 - accuracy: 0.3608
Epoch 5/50
344/344 [==============================] - 23s 67ms/step - loss: 2.2986 - accuracy: 0.3735
Epoch 6/50
344/344 [==============================] - 23s 67ms/step - loss: 2.2363 - accuracy: 0.3833
Epoch 7/50
344/344 [==============================] - 23s 66ms/step - loss: 2.1811 - accuracy: 0.3913
Epoch 8/50
344/344 [==============================] - 23s 67ms/step - loss: 2.1274 - accuracy: 0.4015
Epoch 9/50
344/344 [==============================] - 23s 67ms/step - loss: 2.0722 - accuracy: 0.4099
Epoch 10/50
344/344 [==============================] - 23s 67ms/step - loss: 2.020